In [1]:
# Required for notebooks with asyncio
import nest_asyncio
nest_asyncio.apply()

import asyncio
import json
import sys

print("✅ Imports ready")

✅ Imports ready


In [2]:
from stdio_mcp_client import StdioMCPClient, SyncMCPClient

# Create the async client
async_client = StdioMCPClient()

# Connect to the MCP server (this starts a subprocess)
print("🔌 Connecting to MCP server...")
tools = await async_client.connect(
    command="python3",
    args=["run_mcp_server.py"]
)

print(f"✅ Connected! Discovered {len(tools)} tools:\n")
for tool in tools:
    print(f"   • {tool.name}: {tool.description}")

# Create sync wrapper for easier use
client = SyncMCPClient(async_client)
print("\n🎯 Client ready to use!")

🔌 Connecting to MCP server...
✅ Connected! Discovered 8 tools:

   • read_file: Read the contents of a file. Returns the text content.
   • list_directory: List files and folders in a directory.
   • query_products: Query products database. Filter by category, price range, or search name.
   • query_sales: Query sales data by region, date range, or product.
   • get_analytics: Get analytics: revenue, top_products, sales_by_region, inventory_value.
   • generate_report: Generate and save a markdown report.
   • send_notification: Send notification to slack/email/teams (simulated).
   • create_task: Create a task/todo item.

🎯 Client ready to use!


In [3]:
# Test 1: Query products database
print("🔍 Test: Query Electronics products\n")
result = client.call_tool("query_products", {"category": "Electronics"})
print(result)

🔍 Test: Query Electronics products

[
  {
    "id": 1,
    "name": "Widget Pro",
    "category": "Electronics",
    "price": 299.99,
    "stock": 150
  },
  {
    "id": 2,
    "name": "Gadget Plus",
    "category": "Electronics",
    "price": 199.99,
    "stock": 75
  }
]


In [4]:
# Test 2: Get analytics
print("📊 Test: Get revenue analytics\n")
result = client.call_tool("get_analytics", {"metric": "revenue"})
print(result)

📊 Test: Get revenue analytics

{
  "total_revenue": 23497.8,
  "total_transactions": 6,
  "total_units_sold": 220
}


In [5]:
print("📁 Test: List files directory\n")
result = client.call_tool("list_directory", {"path": "files"})
print(result)

📁 Test: List files directory

[
  {
    "name": "q4_report.txt",
    "type": "file"
  },
  {
    "name": "sales_forecast.txt",
    "type": "file"
  },
  {
    "name": "product_feedback.txt",
    "type": "file"
  },
  {
    "name": "incident_log.txt",
    "type": "file"
  },
  {
    "name": "customer_summary.txt",
    "type": "file"
  },
  {
    "name": "inventory_alert.txt",
    "type": "file"
  }
]


In [6]:
from mcpagentDef import MCPAgent


# Create the agent
agent = MCPAgent(client)
print("🤖 AI Agent created!")
print(f"🔧 Connected to MCP server with {len(client.get_tools_list())} tools")

🔧 Using BedrockBridge for LLM calls
🤖 AI Agent created!
🔧 Connected to MCP server with 8 tools


In [7]:
# Query 1: Simple database query
agent.run("What Electronics products do we have and which is the most expensive?")

🤖 Agent starting...

💭 Thought: I need to query the products database to find all electronics products, then identify which one is the most expensive.

Action: query_products
Action Input: {"category": "Electronics"}
Observation: [
  {
    "id": 1,
    "name": "Widget Pro",
    "category": "Electronics",
    "price": 299.99,
    "stock": 150
  },
  {
    "id": 2,
    "name": "Gadget Plus",
    "category": "Electronics",
    "price": 199.99,
    "stock": 75
  }
]

💭 LLM Response:
Thought: I have the Electronics products from the database. Now I can see what products we have and identify which one is the most expensive by comparing their prices.

Final Answer: We have 2 Electronics products in our inventory:

1. **Widget Pro** - $299.99 (150 units in stock)
2. **Gadget Plus** - $199.99 (75 units in stock)

The most expensive Electronics product is the **Widget Pro** at $299.99.


Final Answer: We have 2 Electronics products in our inventory:

1. **Widget Pro** - $299.99 (150 units in sto

In [ ]:
agent.run("What is our total revenue across all sales?")

In [ ]:
agent.run("Show me the top 3 products by revenue")

In [ ]:
agent.run("What files are in the 'files' directory?")